In [17]:
 # Dependencies
import json
import requests
import pandas as pd
import sqlite3
import datetime

In [18]:
coord_Au = pd.read_csv('Resources/auCoord.csv')
coord = pd.DataFrame(coord_Au)
coord.head()

,City,Latitude,Longtitude,country,iso2,State,capital,Population,population_proper
0,Aberdare,-32.8428,151.3803,Australia,AU,New South Wales,NaN,2473,2473
1,Adelaide,-34.9289,138.6011,Australia,AU,South Australia,admin,1345777,1295714
2,Albany,-35.0228,117.8814,Australia,AU,Western Australia,NaN,34205,29373
3,Albury,-36.0806,146.9158,Australia,AU,New South Wales,NaN,47974,4804
4,Aldgate,-35.0153,138.7360,Australia,AU,South Australia,NaN,3361,3361


In [19]:
weather_AU = pd.read_csv('Resources/weatherAUS.csv')
weather = pd.DataFrame(weather_AU)
weather.head()

,Date,City,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,1/12/2008,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0.0,No
1,2/12/2008,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0.0,No
2,3/12/2008,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,0.0,No
3,4/12/2008,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,1.0,No
4,5/12/2008,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0.2,No


In [20]:
datetime.datetime.strptime(weather['Date'][2], "%d/%m/%Y").strftime("%Y-%m-%d")

'2008-12-03'

In [21]:
pd.to_datetime(weather['Date'],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')

0         2008-12-01
1         2008-12-02
2         2008-12-03
3         2008-12-04
4         2008-12-05
             ...    
226863    2023-03-21
226864    2023-03-22
226865    2023-03-23
226866    2023-03-24
226867    2023-03-25
Name: Date, Length: 226868, dtype: object

In [22]:
weather['Date'] = pd.to_datetime(weather['Date'],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')

In [23]:
joint_df = weather.merge(coord, on='City')
joint_df.head()

,Date,City,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,RISK_MM,RainTomorrow,Latitude,Longtitude,country,iso2,State,capital,Population,population_proper
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,0.0,No,-36.0806,146.9158,Australia,AU,New South Wales,NaN,47974,4804
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,0.0,No,-36.0806,146.9158,Australia,AU,New South Wales,NaN,47974,4804
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,0.0,No,-36.0806,146.9158,Australia,AU,New South Wales,NaN,47974,4804
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,1.0,No,-36.0806,146.9158,Australia,AU,New South Wales,NaN,47974,4804
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,0.2,No,-36.0806,146.9158,Australia,AU,New South Wales,NaN,47974,4804


In [24]:
joint_df_2 = joint_df[['Date', 'City', 'Latitude', 'Longtitude', 'State', 'Population', 'MinTemp','MaxTemp','Rainfall', 'Evaporation', 'Sunshine', 'WindGustDir', 'WindGustSpeed', 'Humidity9am',	'Humidity3pm',	'Pressure9am',	'Pressure3pm',	'Cloud9am',	'Cloud3pm',	'RainToday', 'RISK_MM',	'RainTomorrow']]

In [25]:
selected_au_cities = joint_df_2.loc[joint_df_2['City'].isin(['Perth','Darwin','AliceSprings','Adelaide','Townsville','Brisbane','Sydney','Melbourne','Canberra','Hobart'])]

In [26]:
# selected_au_cities.City.unique()

In [27]:
# from pathlib import Path  
# filepath = Path('c:/Users/user/Desktop/Project3/Resources/AllAus.csv')  
# filepath.parent.mkdir(parents=True, exist_ok=True)  
# joint_df_2.to_csv(filepath)  

In [28]:
conn = sqlite3.connect('AllAus_Weather_History')
c = conn.cursor()

In [29]:
c.execute('create table  IF NOT EXISTS auscityweather( \
wh_id INTEGER PRIMARY KEY AUTOINCREMENT,\
Date varchar(10), \
City varchar(50), \
Latitude double, \
Longtitude double, \
State varchar(30), \
Population varchar(30), \
MinTemp float, \
MaxTemp float, \
Rainfall float, \
Evaporation float, \
Sunshine float, \
WindGustDir varchar(6), \
WindGustSpeed float, \
Humidity9am float, \
Humidity3pm float, \
Pressure9am float, \
Pressure3pm float, \
Cloud9am float, \
Cloud3pm float, \
RainToday boolean, \
RISK_MM float, \
RainTomorrow boolean)')
conn.commit()

In [30]:
selected_au_cities.to_sql('auscityweather', conn, if_exists='replace', index = False)

In [31]:
c.execute('''  
SELECT wh_id,City,Date FROM auscityweather limit 5
          ''')

for row in c.fetchall():
    print (row)

('Sydney', '2008-02-01')
('Sydney', '2008-02-02')
('Sydney', '2008-02-03')
('Sydney', '2008-02-04')
('Sydney', '2008-02-05')
